In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
from datetime import datetime
from typing import Final

import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from torch.utils.tensorboard.writer import SummaryWriter

from tabrel.benchmark.nw_regr import Mlp, MlpConfig, generate_multidim_noisy_data, make_random_r, to_torch, train_nw_arbitrary, NwModelConfig, RelNwRegr




def test_mlp() -> None:
    housing = fetch_california_housing()
    x, y = to_torch(housing.data), to_torch(housing.target)
    mlp = Mlp(MlpConfig(in_dim=x.shape[1], hidden_dim=64, out_dim=8, dropout=.1))
    print(mlp(x).shape)

test_mlp()

In [ ]:
n_samples: Final[int] = 300
seed: Final[int] = 42
x_dim: Final[int] = 6
n_epochs: Final[int] = 5000

# mse, r2, _, _ = train_nw_arbitrary(
#     x_backgnd=x_np[back_ids],
#     y_backgnd=y_np[back_ids],
#     x_query=x_np[query_ids],
#     y_query=y_np[query_ids],
#     x_val=x_np[val_ids],
#     y_val=y_np[val_ids],
#     r_query_backgnd=r[query_ids][:, back_ids],
#     r_val_nonval=r[val_ids][:, train_ids],
#     cfg=NwModelConfig(input_dim=x_dim, trainable_weights_matrix=True,),
#     lr=1e-3,
#     n_epochs=n_epochs
# )

# print(mse, r2)

In [ ]:
def train_nw_mlp(
        seed: int,
        mlp_hid_dim: int,
        mlp_out_dim: int,
        dropout: float,
        weight_decay: float,
        writer: SummaryWriter | None,
        trainable_weights: bool = False, # TODO try True
        _n_epochs: int = n_epochs,
        ) -> tuple[float, float]:
    x_np, y_np, c = generate_multidim_noisy_data(n_samples, n_clusters=3, x_dim=x_dim, seed=seed)
    r = make_random_r(seed, c)

    n_query = n_back = n_samples // 3
    n_train = n_query + n_back
    n_val = n_samples - n_train
    back_ids = np.arange(n_back)
    query_ids = np.arange(n_query) + n_back
    train_ids = np.arange(n_train)
    val_ids = np.arange(n_val) + n_train

    x_torch, y_torch = to_torch(x_np), to_torch(y_np)
    x_train, y_train = x_torch[train_ids], y_torch[train_ids]
    x_mean, x_std = x_train.mean(dim=0), x_train.std(dim=0)
    x_norm = (x_torch - x_mean) / (x_std + 1e-8)

    x_b, y_b = x_norm[back_ids], y_torch[back_ids]
    x_q, y_q = x_norm[query_ids], y_torch[query_ids]
    x_v, y_v = x_norm[val_ids], y_torch[val_ids]

    r_q_b = to_torch(r)[query_ids][:, back_ids]
    r_val_nval = to_torch(r)[val_ids][:, train_ids]

    torch.random.manual_seed(seed)
    config = NwModelConfig(
        input_dim=mlp_out_dim,
        init_sigma=1.,
        init_r_scale=1.,
        trainable_weights_matrix=trainable_weights,
        mlp_config=MlpConfig(
            in_dim=x_dim,
            hidden_dim=mlp_hid_dim,
            out_dim=mlp_out_dim,
            dropout=dropout,
        )
    )
    model = RelNwRegr(config)
    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=weight_decay)
    loss_fn = torch.nn.MSELoss()
    last_val_r2: float | None = None
    last_val_mse: float | None = None
    model.train()
    for epoch in tqdm(range(_n_epochs)):
        optimizer.zero_grad()
        y_pred = model(x_b, y_b, x_q, r_q_b)
        loss = loss_fn(y_pred, y_q)
        loss.backward()
        optimizer.step()
        if writer:
            writer.add_scalar("train/loss", loss.item(), epoch)

        if epoch % 20 != 0: continue

        model.eval()
        with torch.no_grad():
            y_v_pred = model(
                x_norm[train_ids],
                y_train,
                x_v,
                r_val_nval,
            )
            y_v_pred_np = y_v_pred.numpy()
            y_v_np = y_v.numpy()

        last_val_mse = mean_squared_error(y_v_np, y_v_pred_np)
        last_val_r2 = r2_score(y_v_np, y_v_pred_np)
        if writer:
            writer.add_scalar("val/mse", last_val_mse, epoch)
            writer.add_scalar("val/r2", last_val_r2, epoch)
        
        model.train()
    
    if last_val_r2 is None:
        raise ValueError("last_val_r2 is None, probably not enough iterations")
    return last_val_mse if last_val_mse is not None else float('inf'), last_val_r2 if last_val_r2 is not None else -float('inf')


In [ ]:
log_dir: Final[str] = "tb_logs" + datetime.isoformat(datetime.now()).replace(":", "_")
print(log_dir)

In [ ]:
# %tensorboard --logdir {log_dir}

# writer = SummaryWriter(log_dir=log_dir)
# train_nw_mlp(
#     seed=seed,
#     mlp_hid_dim=64,
#     mlp_out_dim=6,
#     dropout=.2,
#     weight_decay=1e-4,
#     writer=writer)

mses, r2s = [], []
for _seed in range(15):
    # mse, r2 = train_nw_mlp(
    #     seed=_seed,
    #     weight_decay=0.0006913422,
    #     mlp_hid_dim=8,
    #     mlp_out_dim=20,
    #     dropout=0.09302061,
    #     writer=None,
    # )
    # 0.23309 var 0.00074
    # 0.81283 var 0.00048
    mse, r2 = train_nw_mlp(
        seed=_seed,
        weight_decay=0.002,
        mlp_hid_dim=10,
        mlp_out_dim=20,
        dropout=.19,
        _n_epochs=n_epochs,
        trainable_weights=False,
        writer=None,
    )
    # 0.23555 var 0.00118
    # 0.81012 var 0.00048
    mses.append(mse)
    r2s.append(r2)
    



for metrics in (mses, r2s):
    print(f"{np.mean(metrics):.5f} var {np.var(metrics):.5f}")

In [ ]:
import optuna

def objective(trial: optuna.Trial) -> float:
    weight_decay = trial.suggest_float("weight_decay", 0., 1e-2)
    mlp_hid_dim = trial.suggest_int("mlp_hid_dim", 6, 100)
    mlp_out_dim = trial.suggest_int("mlp_out_dim", 1, 20)
    dropout = trial.suggest_float("dropout", 0., .5)
    val_r2 = train_nw_mlp(
        seed=seed,
        mlp_hid_dim=mlp_hid_dim,
        mlp_out_dim=mlp_out_dim,
        dropout=dropout,
        weight_decay=weight_decay,
        writer=None,
    )[1]
    return val_r2

study = optuna.create_study(direction="maximize", study_name=f"r_random_{datetime.now()}", storage="sqlite:///db.sqlite3")
study.optimize(objective)

code for loading an existing trial

In [ ]:
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///db.sqlite3",
    study_name="r_deterministic_2025-11-07 14:52:56.356900",
    load_if_exists=True,
)

# study.optimize(objective, n_trials=10)
print(f"Best trial: {study.best_trial}")


In [ ]:
study.best_params

In [ ]:
%tensorboard --logdir {log_dir} 

writer = SummaryWriter(log_dir=log_dir)
train_nw_mlp(
    seed=seed,
    mlp_hid_dim=study.best_params["mlp_hid_dim"],
    mlp_out_dim=study.best_params["mlp_out_dim"],
    dropout=study.best_params["dropout"],
    weight_decay=study.best_params["weight_decay"],
    writer=writer,
    _n_epochs = 10_000
)

In [ ]:
# TODO transform Y (with another MLP?)
# TODO embeddings AND learnable norm (+ weight decay?)
# TODO try RTDL num embeddings instead of or before perceptrons
# TODO another synthetic?